In [3]:
import pandas as pd
import folium

# 엑셀 파일에서 데이터 읽어오기 (주문 데이터)
order_file_path = '서울 마포_0712_4번이상.xlsx'
order_df = pd.read_excel(order_file_path)

# 엑셀 파일에서 데이터 읽어오기 (주차구역 데이터)
parking_file_path = '마포_배치존.xlsx'
parking_df = pd.read_excel(parking_file_path)

# 좌표 데이터를 리스트 형태로 변환하는 함수 정의
def parse_coordinates(coordinates_str):
    try:
        coordinates = eval(coordinates_str)
        return [(coord["lat"], coord["lng"]) for coord in coordinates]
    except:
        return None

# 각 주문 ID에 대해 좌표 데이터 추출하여 딕셔너리로 저장
order_coordinates = {}
for idx, row in order_df.iterrows():
    order_id = row['order_id']
    coordinates_str = row['coordinates']
    coords_list = parse_coordinates(coordinates_str)
    if coords_list:
        order_coordinates[order_id] = coords_list

# 지도 객체 생성
m = folium.Map(location=[37.5, 126.9], zoom_start=12)  # 초기 지도 위치와 확대 정도 설정

# FeatureGroup 생성
start_group = folium.FeatureGroup(name='출발지점')
end_group = folium.FeatureGroup(name='도착지점')
parking_group = folium.FeatureGroup(name='주차구역')

# 주문 ID에 따라 선 그리기
for order_id, coords in order_coordinates.items():
    folium.PolyLine(locations=coords, color='blue', weight=2.5, opacity=1).add_to(m)

# 시작과 종료 좌표 점 형식으로 추가하기
for idx, row in order_df.iterrows():
    start_lat, start_lng = row['start_lat'], row['start_lng']
    end_lat, end_lng = row['end_lat'], row['end_lng']

    # 시작과 종료 지점 점 형식으로 추가
    folium.CircleMarker(location=[start_lat, start_lng], radius=5, color='green', fill=True, fill_color='green').add_to(start_group)
    folium.CircleMarker(location=[end_lat, end_lng], radius=5, color='red', fill=True, fill_color='red').add_to(end_group)

# 주차구역 좌표 마커 형식으로 추가하기
for idx, row in parking_df.iterrows():
    name = row['name']
    lat, lng = row['lat'], row['lng']

    # 주차구역 마커 형식으로 추가
    folium.Marker(location=[lat, lng], icon=folium.Icon(color='purple', icon='ok-sign', prefix='fa'), popup=name).add_to(parking_group)

# FeatureGroup 추가
start_group.add_to(m)
end_group.add_to(m)
parking_group.add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 지도를 HTML 파일로 저장하여 시각화
m
